In [9]:
import os
import re
from collections import Counter
import subprocess
from pathlib import Path

import pandas as pd

import mgitools.os_helpers as os_helpers
import mgitools.bsub as bsub

## set up directory structure from bammap

In [34]:
outer_dir = '/gscmnt/gc2686/rna_editing/TCGA-TGCT/somaticwrapper_50/'
run_dir = os.path.join(outer_dir, 'run_dir')

somaticwrapper_log_dir = os.path.join(outer_dir, 'logs')
reference_fp = '/gscmnt/gc2686/rna_editing/data/reference/gdc/GRCh38.d1.vd1.fa'

for fp in [outer_dir, run_dir, somaticwrapper_log_dir]:
    Path(fp).mkdir(exist_ok=True, parents=True)

In [35]:
bammap = pd.read_csv('../TCGA-TGCT.bammap', sep='\t')
bammap

,# sample_name,case,disease,experimental_strategy,sample_type,data_path,filesize,data_format,reference,UUID,system
0,TCGA-XE-AAO7.WXS.T.hg38,TCGA-XE-AAO7,TGCT,WXS,tumor,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/0264...,62559578473,BAM,hg38,02645a88-2b1b-4390-8751-8d11681b138a,MGI
1,TCGA-YU-A90R.WXS.T.hg38,TCGA-YU-A90R,TGCT,WXS,tumor,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/02a7...,44291562018,BAM,hg38,02a7a293-24ab-42e7-b7ef-4ee53c76da8c,MGI
2,TCGA-SN-A84Y.WXS.N.hg38,TCGA-SN-A84Y,TGCT,WXS,blood_normal,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/0345...,17000519818,BAM,hg38,03451dc0-4fd1-4718-a1c1-4b4bfa5efeac,MGI
3,TCGA-2G-AAFG.WXS.N.hg38,TCGA-2G-AAFG,TGCT,WXS,blood_normal,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/070a...,12729573440,BAM,hg38,070ad00f-9136-4ada-950f-ede6bc641f15,MGI
4,TCGA-2G-AAGO.WXS.N.hg38,TCGA-2G-AAGO,TGCT,WXS,blood_normal,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/07d8...,13320338881,BAM,hg38,07d849ad-90b8-4067-8b45-e62a72514df5,MGI
5,TCGA-XE-A8H5.WXS.N.hg38,TCGA-XE-A8H5,TGCT,WXS,blood_normal,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/07e9...,14423250406,BAM,hg38,07e9af59-1016-45f3-bc5b-7d6c09c7624f,MGI
6,TCGA-XE-AAOB.WXS.N.hg38,TCGA-XE-AAOB,TGCT,WXS,blood_normal,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/08cc...,15353296172,BAM,hg38,08cc4424-aa6d-4f6e-93d5-02d7a257c532,MGI
7,TCGA-2G-AAF0.WXS.T.hg38,TCGA-2G-AAF0,TGCT,WXS,tumor,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/095d...,41723582182,BAM,hg38,095d0d8f-cddf-47b1-ac22-1c75d7ec6870,MGI
8,TCGA-YU-A90S.WXS.N.hg38,TCGA-YU-A90S,TGCT,WXS,blood_normal,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/0a2a...,17097528382,BAM,hg38,0a2a49d5-0a15-4378-ab1b-05495f4e015b,MGI
9,TCGA-2G-AAIE.WXS.N.hg38,TCGA-2G-AAIE,TGCT,WXS,blood_normal,/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/0ac8...,39055441574,BAM,hg38,0ac8fb26-062f-44f3-9333-7699fe645c05,MGI


In [36]:
case_to_bams = {}
for i, row in bammap.iterrows():
    if row['case'] not in case_to_bams:
        case_to_bams[row['case']] = {}
    case_to_bams[row['case']][row['sample_type']] = row['data_path']
case_to_bams = {k:v for k, v in case_to_bams.items() if len(v)==2}
len(case_to_bams), sorted(case_to_bams.items())[:5]
        

(108,
 [('TCGA-2G-AAER',
   {'blood_normal': '/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/4284dd7c-c9ce-4bc4-a98e-2ebaee4b26e4/440bbe51-35bb-43ef-bb89-1c75eb34c16a_wxs_gdc_realn.bam',
    'tumor': '/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/50648672-1f25-4f37-99d2-55f7b34f3a60/f77d49ed-575b-40cd-b2e9-8413c33c1694_wxs_gdc_realn.bam'}),
  ('TCGA-2G-AAES',
   {'tumor': '/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/483b4b01-80fd-4b28-aa82-1da191676913/da323434-58be-4479-ae68-5bab26f7a1eb_wxs_gdc_realn.bam',
    'blood_normal': '/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/62a2056d-2c0b-4d67-a2a5-e794f87e08af/91dcaee8-ba9e-4c97-b4a5-0859293c711a_wxs_gdc_realn.bam'}),
  ('TCGA-2G-AAEW',
   {'blood_normal': '/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/5b1149fa-359e-44b1-9356-68f71e173da1/TCGA-2G-AAEW-10A-01D-A433-10_Illumina_gdc_realn.bam',
    'tumor': '/gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/f54698a3-c2b7-4a4f-a11b-8b20ae586d40/TCGA-2G-AAEW-01A-11D-A42Y-10_Illumina_gdc_realn.bam'}),
  ('TCGA-2G-AAEX

In [37]:
case_to_bams = {k:case_to_bams[k] for k in list(case_to_bams.keys())[:50]}
len(case_to_bams)

50

In [38]:
## set up dir via symlink
commands = []
for case, fp_dict in case_to_bams.items():
    case_dir = os.path.join(run_dir, case)
    Path(case_dir).mkdir(exist_ok=True, parents=True)
    
    ## do normal
    fp = fp_dict['blood_normal']
    new_fp = os.path.join(case_dir, f'{case}.N.bam')
    commands.append(f'ln -s {fp} {new_fp}')
    ## do tumor
    fp = fp_dict['tumor']
    new_fp = os.path.join(case_dir, f'{case}.T.bam')
    commands.append(f'ln -s {fp} {new_fp}')
    
bsub.generate_bsub_bash_script(commands, 'python:3.7', os.path.join(outer_dir, 'symlink_commands.sh'),
                          min_memory=4, max_memory=5)

commands[:4]

['ln -s /gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/d5979bfc-cd27-4e8f-8db0-26430477d943/a1aa73dd-48a1-4ab8-ace0-fd78dd75a762_wxs_gdc_realn.bam /gscmnt/gc2686/rna_editing/TCGA-TGCT/somaticwrapper_50/run_dir/TCGA-XE-AAO7/TCGA-XE-AAO7.N.bam',
 'ln -s /gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/02645a88-2b1b-4390-8751-8d11681b138a/5a0b350a-b4b1-499f-b7cd-a40923b6534d_wxs_gdc_realn.bam /gscmnt/gc2686/rna_editing/TCGA-TGCT/somaticwrapper_50/run_dir/TCGA-XE-AAO7/TCGA-XE-AAO7.T.bam',
 'ln -s /gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/f18fe04b-f74d-42ce-8033-270021987a52/0482b17c-a7cb-425a-be64-6e5f3de6e4d3_wxs_gdc_realn.bam /gscmnt/gc2686/rna_editing/TCGA-TGCT/somaticwrapper_50/run_dir/TCGA-YU-A90R/TCGA-YU-A90R.N.bam',
 'ln -s /gscmnt/gc2686/rna_editing/TCGA-TGCT/bams/02a7a293-24ab-42e7-b7ef-4ee53c76da8c/ce9e2239-0113-47f6-a42e-af4e40a1323e_wxs_gdc_realn.bam /gscmnt/gc2686/rna_editing/TCGA-TGCT/somaticwrapper_50/run_dir/TCGA-YU-A90R/TCGA-YU-A90R.T.bam']

In [ ]:
## index directory structure

In [39]:
fps = sorted(os_helpers.listfiles(run_dir, regex='.bam$'))
len(fps), fps[:4]

(100,
 ['/gscmnt/gc2686/rna_editing/TCGA-TGCT/somaticwrapper_50/run_dir/TCGA-2G-AAER/TCGA-2G-AAER.N.bam',
  '/gscmnt/gc2686/rna_editing/TCGA-TGCT/somaticwrapper_50/run_dir/TCGA-2G-AAER/TCGA-2G-AAER.T.bam',
  '/gscmnt/gc2686/rna_editing/TCGA-TGCT/somaticwrapper_50/run_dir/TCGA-2G-AAEX/TCGA-2G-AAEX.N.bam',
  '/gscmnt/gc2686/rna_editing/TCGA-TGCT/somaticwrapper_50/run_dir/TCGA-2G-AAEX/TCGA-2G-AAEX.T.bam'])

In [40]:
def get_index_command(bam_fp):
    return f'samtools index {bam_fp}'

In [41]:
commands = [f'samtools index {fp}' for fp in fps]
bsub.generate_bsub_bash_script(commands, 'estorrs/slate:0.0.2', os.path.join(outer_dir, 'index_commands.sh'),
                          min_memory=10, max_memory=11)

## generate sommaticwrapper command

In [ ]:
perl somaticwrapper.pl --srg --step --sre --rdir --ref --log --q --mincovt --mincovn --minvaf --maxindsize --exonic


In [42]:
command = f'perl somaticwrapper.pl --rdir {run_dir} --log {somaticwrapper_log_dir} --step 0 \
--ref {reference_fp} -q research-hpc'
f = open(os.path.join(outer_dir, 'somaticwrapper_start.sh'), 'w')
f.write(command + '\n')
f.close()